## Лабораторная работа №4. Определение части речи слов.
### Выполнила: Залесская Галина, 16ПМИ


In [1]:
import nltk
from sklearn.model_selection import train_test_split 
from lxml import etree

In [44]:
word_corpus = []
path = '/home/kotik/code/NLP2/my_labs/tagged_texts/'
end = 833
for index in range(1, end):
    file = open(path+str(index)+'.txt', 'r').read().split("# sent_id = ")
    for sentence in file[1:]:
        words = sentence.split('\n') 
        sentence_info = []
        for word in words[2:]:
            word_info = word.split('\t')
            if len(word_info)>4 and word_info[1] != '"':
                sentence_info.append((word_info[1], word_info[3]))
        word_corpus.append(sentence_info)
print(len(word_corpus), len(sentence_info))

8241 4


Word_corpus - лист из 8241 предложений, представленных в виде листов таплов (слово, часть речи). Суммарно имеем 148 000 слов (включая пунктуации).

In [46]:
train, test = train_test_split(word_corpus, test_size=0.2, random_state=42)

In [47]:
train[0]

[('Таким', 'DET'),
 ('мы', 'PRON'),
 ('увидели', 'VERB'),
 ('Ральфа', 'PROPN'),
 ('Шмульке', 'PROPN'),
 (',', 'PUNCT'),
 ('лидера', 'NOUN'),
 ('движения', 'NOUN'),
 ('за', 'ADP'),
 ('восстановление', 'NOUN'),
 ('Восточной', 'ADJ'),
 ('Германии', 'PROPN'),
 ('в', 'ADP'),
 ('качестве', 'NOUN'),
 ('отдельного', 'ADJ'),
 ('государства', 'NOUN'),
 ('.', 'PUNCT')]

In [5]:
# nltk.download()
# maxent_ne_chunker
# words
# averaged_perceptron_tagger
# universal_tagset


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [52]:
nltk.pos_tag([word for (word, tag) in word_corpus[0]], tagset='universal')

[('Таким', 'ADJ'),
 ('мы', 'NOUN'),
 ('увидели', 'NOUN'),
 ('Ральфа', 'NOUN'),
 ('Шмульке', 'NOUN'),
 (',', '.'),
 ('лидера', 'NOUN'),
 ('движения', 'NOUN'),
 ('за', 'NOUN'),
 ('восстановление', 'NOUN'),
 ('Восточной', 'NOUN'),
 ('Германии', 'NOUN'),
 ('в', 'NOUN'),
 ('качестве', 'NOUN'),
 ('отдельного', 'NOUN'),
 ('государства', 'NOUN'),
 ('.', '.')]

In [57]:
correct = 0
for sentence in word_corpus[1:10]:
    for [word, tag] in sentence:
        predicted = nltk.pos_tag([word], tagset='universal')[0][1]
        predicted = "PUNCT" if predicted=='.' else predicted
        print(predicted, end=' ')
        correct += (tag==predicted)
print('\n', correct / sum([len(sent) for sent in word_corpus[1:10]]))

NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NUM NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NUM NOUN NUM NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN NOUN NOUN NOUN PUNCT NOUN NOUN ADJ NOUN NOUN NOUN NOUN NOUN NOUN

NLTK предпочитает тегировать все слова как существительные. И правильно определять рунктуацию, потому что это легко. Поэтому мы иеем такую низкую точность, вытягиваемую только за счет частого появления существительных и правильной разметки пунктуации.

Очень полезная функция)

In [61]:
unigram_tagger = nltk.UnigramTagger(train)
unigram_tagger.evaluate(test)

0.8356479913285008

Теггер на основе юниграм увеличил точность предсказывания почти в 2 раза!

In [64]:
threegram_gram = nltk.TrigramTagger(train)
threegram_gram.evaluate(test)

0.12058803604091864

Теггер на основе триграмм дает довольно плохой результат, что, скорее всего, связано с маленьким размером предложений. Попробуем комбинацию теггеров из триграмм, биграмм и юниграмм, в этом случае предсказание будет вычисляться на основ 3 близких слов, если это не помогло, то 2 и, наконец, одного.

In [65]:
unigram_tagger = nltk.UnigramTagger(train)
bigram_tagger = nltk.BigramTagger(train, backoff=unigram_tagger)
threegram_gram = nltk.TrigramTagger(train, backoff=bigram_tagger)
threegram_gram.evaluate(test)

0.8388998035363457

Результат получился немного лучше. Ожидается, если мы добавим дефолтный тег, который будет выбираться, если предсказания н-грамм не помогли, то можно увеличить точность. В качестве дефолтного тега выберем существительные, так как их больше, и их нелегко определить.

In [71]:
default = nltk.DefaultTagger('NOUN') 
unigram_tagger = nltk.UnigramTagger(train, backoff=default)
bigram_tagger = nltk.BigramTagger(train, backoff=unigram_tagger)
threegram_gram = nltk.TrigramTagger(train, backoff=bigram_tagger)
threegram_gram.evaluate(test)

0.8931305467109274

Результат и правда стал еще лучше

## Тегирование новостного датасета

In [76]:
import spacy
from spacy import displacy

In [83]:
news_df = open('news.xml').read()

root = etree.fromstring(news_df)
catalog = []

for element_lvl1 in root:
    news = ''
    for element_lvl2 in element_lvl1:
        if element_lvl2.text is not None:
            news += element_lvl2.text + '\n'
    catalog.append(news)

In [78]:
nlp = spacy.load('en_core_web_sm')

In [85]:
docs = [nlp(txt) for txt in catalog]

In [93]:
POS_tagging = []
doc = docs[0]
for token in doc:
    POS_tagging.append((token.text, token.pos_))

POS_tagging

[('uk', 'PROPN'),
 ('\n', 'SPACE'),
 ('Sturgeon', 'PROPN'),
 ('agrees', 'VERB'),
 ('to', 'ADP'),
 ('wider', 'ADJ'),
 ('review', 'NOUN'),
 ('of', 'ADP'),
 ('Scottish', 'ADJ'),
 ('education', 'NOUN'),
 ('\n', 'SPACE'),
 ('Image', 'NOUN'),
 ('copyright', 'NOUN'),
 ('PA', 'NOUN'),
 ('Media', 'NOUN'),
 ('Image', 'NOUN'),
 ('caption', 'NOUN'),
 ('Nicola', 'PROPN'),
 ('Sturgeon', 'PROPN'),
 ('said', 'VERB'),
 ('her', 'DET'),
 ('government', 'NOUN'),
 ('would', 'VERB'),
 ('accept', 'VERB'),
 ('the', 'DET'),
 ('result', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('vote', 'NOUN'),
 ('despite', 'SCONJ'),
 ('her', 'DET'),
 ('SNP', 'PROPN'),
 ('opposing', 'VERB'),
 ('it', 'PRON'),
 ('\n\n', 'SPACE'),
 ('Nicola', 'PROPN'),
 ('Sturgeon', 'PROPN'),
 ('has', 'AUX'),
 ('agreed', 'VERB'),
 ('to', 'ADP'),
 ('a', 'DET'),
 ('wider', 'ADJ'),
 ('inquiry', 'NOUN'),
 ('into', 'ADP'),
 ('Scottish', 'ADJ'),
 ('education', 'NOUN'),
 ('after', 'ADP'),
 ('opposition', 'NOUN'),
 ('MSPs', 'NOUN'),
 ('united', 'VERB'),

Определение частей речи новостного датасета выглядит весьма правдоподобным

In [95]:
displacy.render(doc, jupyter=True)

Этот график позволяет увидеть характер влияния слов друг на друга, а так же лучше рассмотреть тегированные части речи. Но  из-за большого количества слов, график становится едва читаемым.

In [97]:
displacy.render(doc, style='ent', jupyter=True)

Можно говорить, что новость протегировалась хорошо. Как минимум, теггер понял из статьи явно больше, чем я, потому что для меня она совершенно нечитаемая. Единственная замеченная ошибка, что в начале "Education" неправильно помечен как организация.